E VS I CLASSIFICATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "E_vs_I_data.csv")
df = df.dropna(how='any',axis=0)
df.sample(10)

,label,text
73327,0,If you were an ISFP you'd possess emotional in...
196811,1,I have a fb but I find it so ridiculously usel...
126725,1,Retook this cause I was 1) bored 2) curious to...
140182,1,"Ha, than we have opposite views, as I see tan ..."
86806,0,"That sounds like a lower, annoying Ne to me? M..."
195042,1,"See, if I don't really like someone, I honestl..."
54097,0,1. I don't end up in leadership positions that...
103786,1,Have you read ''Wicked'' by Gregory Maguire? I...
95048,0,ENTP's lead to the coup
165380,1,I usually use this informing style when I don ...


In [3]:
DOWNSAMPLE = 30000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

0    15018
1    14982
Name: label, dtype: int64

In [4]:
df_balanced.head(0)

,label,text


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [6]:
train.head(3)

,label,text
196859,1,"I'm ISTP - living in the moment, always busy, ..."
191684,1,In their playful moments Battler and Maria see...
198123,1,"Best of luck to you! To echo, I think it's imp..."


In [7]:
test.head(3)

,label,text
23119,0,Ambivert Ntuitive Thinker Introvert Sensor Ex...
16213,0,Ti looks inside for processing thoughts. Te lo...
98001,0,I think we're all graced with ridiculously goo...


In [8]:
ourpur_dir = binary_data_path + "EvsI_BERT/"
train.to_csv(ourpur_dir+"E_vs_I_train_data.csv", index=False)
test.to_csv(ourpur_dir+"E_vs_I_test_data.csv", index=False)

In [9]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 71.9 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 212 kB 73.5 MB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [10]:
data_files = {"train": ourpur_dir+"E_vs_I_train_data.csv", "test": ourpur_dir+"E_vs_I_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')



Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-af85dd8cb006158c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4500
    })
})

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 59.5 MB/s 


In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [15]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

4500

In [16]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/25500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [17]:
tokenized_train[0]

{'label': 0,
 'text': 'This is an honest guy right here. I would take every other post here claiming a 130+ IQ with a bucket of salt.    Sees past the ego of making an inflated claim and questions the validity of the...',
 'input_ids': [101,
  2023,
  2003,
  2019,
  7481,
  3124,
  2157,
  2182,
  1012,
  1045,
  2052,
  2202,
  2296,
  2060,
  2695,
  2182,
  6815,
  1037,
  7558,
  1009,
  26264,
  2007,
  1037,
  13610,
  1997,
  5474,
  1012,
  5927,
  2627,
  1996,
  13059,
  1997,
  2437,
  2019,
  29561,
  4366,
  1998,
  3980,
  1996,
  16406,
  1997,
  1996,
  1012,
  1012,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [20]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [21]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [22]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7970
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.682100
1000,0.672000
1500,0.665700
2000,0.633300
2500,0.611100
3000,0.603600
3500,0.530800
4000,0.478200
4500,0.474600
5000,0.425000


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/EvsI_BERT/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/D

TrainOutput(global_step=7970, training_loss=0.47236958196202067, metrics={'train_runtime': 943.4184, 'train_samples_per_second': 135.147, 'train_steps_per_second': 8.448, 'total_flos': 1989153226052640.0, 'train_loss': 0.47236958196202067, 'epoch': 5.0})

In [23]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 16


<ipython-input-20-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 1.071434736251831,
 'eval_accuracy': 0.6255555555555555,
 'eval_f1': 0.6244706931134387,
 'eval_runtime': 9.2302,
 'eval_samples_per_second': 487.529,
 'eval_steps_per_second': 30.552,
 'epoch': 5.0}